## Asynchronous scraping

In [1]:
pip install aiohttp


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install asyncio


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import re

In [4]:
!pip install nest-asyncio 
import nest_asyncio
nest_asyncio.apply()


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [17]:
async def scrape_save_links(text):
  soup = BeautifulSoup(text, 'html.parser')
  file = open('csv_file_S', 'a', newline='')
  writer = csv.writer(file, delimiter=',' )
  for link in soup.findAll('a', attrs={'href': re.compile('^http')}):
    link = link.get('href')
    writer.writerow([link])
  file.close()

In [18]:
async def fetch(session, url):
  try:
    async with session.get(url) as response:
      text = await response.text()
      task = asyncio.create_task(scrape_save_links(text))
      await task
  except Exception as e:
    print(str(e))

In [19]:
# manage multiple urls so we can fetch and process them simultaneously
async def scrape(urls):
  tasks = []
  async with aiohttp.ClientSession() as session:
    for url in urls:
      tasks.append(fetch(session, url))
    await asyncio.gather(*tasks)

In [20]:
urls = ['https://analytics.usa.gov/', 'https://www.python.org/', 'https://pandas.pydata.org/']
asyncio.run(scrape(urls=urls))